In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
root_folder = os.path.dirname(os.getcwd())
sys.path.append(root_folder)
print(os.getcwd())

/Users/lingjiekong/Documents/github/cambioml/plotano/plotano


In [3]:
from plotano.db.ranking_database import RankingDatabase

In [4]:

db = RankingDatabase(
    # db_file = "/Users/lingjiekong/Documents/github/cambioml/plotano/plotano/db/qa.db",
    debug=True)

Table contents after creating table:


In [5]:
db.create_table()

Table contents after creating table:


In [6]:
id = db.insert_ranking(question="Who is Rachel?", answer="CEO of CambioML", ranking=1)
id = db.insert_ranking(question="Who is Jared?", answer="CTO of CambioML", ranking=2)
id = db.insert_ranking(question="Who is Kimi?", answer="Nobody of CambioML", ranking=2)

Table contents after inserting entry:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Ranking: 1
Table contents after inserting entry:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Ranking: 1
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Ranking: 2
Table contents after inserting entry:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Ranking: 1
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Ranking: 2
ID: 3, Question: Who is Kimi?, Answer: Nobody of CambioML, Ranking: 2


In [7]:
my_sql_data = db.retrieve_all_question_answers()
my_sql_data

[(1, 'Who is Rachel?', 'CEO of CambioML', 1),
 (2, 'Who is Jared?', 'CTO of CambioML', 2),
 (3, 'Who is Kimi?', 'Nobody of CambioML', 2)]

In [8]:
header = ('ID', 'Question', 'Answer', 'ranking')

common_header = ['ID', 'Question', 'Answer']


In [9]:
import csv
header = ('ID', 'Question', 'Answer', 'ranking')

my_sql_data_path = 'my_sql_data.csv'
with open(my_sql_data_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(my_sql_data)

In [8]:
# db.close_connection()

### Load my_data to RLHF

In [10]:
import pandas as pd
my_data_hf = pd.read_csv(my_sql_data_path)[common_header]
my_data_hf

,ID,Question,Answer
0,1,Who is Rachel?,CEO of CambioML
1,2,Who is Jared?,CTO of CambioML
2,3,Who is Kimi?,Nobody of CambioML


In [14]:
# !pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.4 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 106.6 MB/s eta 0:00:00


In [8]:

stackoverflow_dataset = pd.read_parquet('/home/ubuntu/datasets/train-00000-of-00020.parquet', engine='fastparquet', index=False)
stackoverflow_dataset.head()

,qid,question,date,metadata,response_j,response_k
0,12891264,I am using jQuery fileupload plugin and I want...,2012/10/15,"[https://Stackoverflow.com/questions/12891264,...","Looking at the library code, seems all events ...",Check if the server-side uploading script retu...
1,588860,When reading about the Big Bang you’ll commonl...,2020/10/22,[https://physics.stackexchange.com/questions/5...,No. The local energy *density* is reduced as t...,Are you aware of redshift? If a photon with a ...
2,141736,I've got a flight back to my own country with ...,2019/07/09,[https://travel.stackexchange.com/questions/14...,12:45 pm is also 12:45 o'clock in internationa...,PM is an acronym for the Latin *post meridiem*...
3,1484683,I have a piece of code that works fine in the ...,2009/09/27,"[https://Stackoverflow.com/questions/1484683, ...",I had this exact same problem. The only way I ...,"In your console, the rails environment is load..."
4,1484683,I have a piece of code that works fine in the ...,2009/09/27,"[https://Stackoverflow.com/questions/1484683, ...",I had this exact same problem. The only way I ...,Did you tell rake that your task is dependent ...


In [9]:
my_stackoverflow_dataset = stackoverflow_dataset[['qid', 'question', 'response_j']]
my_stackoverflow_dataset.head()

,qid,question,response_j
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ..."
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...
4,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...


In [10]:
my_stackoverflow_dataset.columns = common_header
my_stackoverflow_dataset.head()

,ID,Question,Answer
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ..."
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...
4,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...


In [11]:
my_data_hf = pd.concat([my_data_hf, my_stackoverflow_dataset])


In [12]:
my_data_hf.head(10)

,ID,Question,Answer
0,1,Who is Rachel?,CEO of CambioML
1,2,Who is Jared?,CTO of CambioML
2,3,Who is Kimi?,Nobody of CambioML
3,4,Who is Rachel?,CEO of CambioML
4,5,Who is Jared?,CTO of CambioML
5,6,Who is Kimi?,Nobody of CambioML
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ..."
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...


In [22]:
my_data_hf.head(20).to_csv(my_sql_data_path)

### Load to huggingface

In [23]:
from datasets import load_dataset
ds = load_dataset('csv', data_files=my_sql_data_path)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1514.19it/s]


Dataset csv downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/csv/default-3aaa395a57a72ad0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 859.66it/s]


In [24]:
ds = ds['train'] # Convert DatasetDict to Dataset

ds = ds.train_test_split(test_size=0.1)
ds


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ID', 'Question', 'Answer'],
        num_rows: 18
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ID', 'Question', 'Answer'],
        num_rows: 2
    })
})

In [26]:
train_dataset = ds["train"]
test_dataset = ds["test"]

In [27]:
train_dataset, test_dataset

(Dataset({
     features: ['Unnamed: 0', 'ID', 'Question', 'Answer'],
     num_rows: 18
 }),
 Dataset({
     features: ['Unnamed: 0', 'ID', 'Question', 'Answer'],
     num_rows: 2
 }))

### Test RLHF code with DB

In [30]:
from plotano.rlhf.rlhf import RLHFConfig


config = RLHFConfig(
    base_model_path="elinas/llama-7b-hf-transformers-4.29", ## "distilroberta-base", ##
    reward_model_path="databricks/dolly-v2-3b", 
    dataset_type="csv", ## "huggingface", ## 
    dataset_name='/home/ubuntu/plotano/plotano/my_sql_data.csv',
    max_steps=3,
    output_dir="../tests/",
    dataset_subset_reward_train=100,
)

In [33]:
from plotano.rlhf.rlhf import SFT ##, RewardTrainer, RL

rlhf_step1_sft = SFT(config)

Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-3aaa395a57a72ad0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 923.86it/s]
Loading cached split indices for dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-3aaa395a57a72ad0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-d350c74ac0a771de.arrow and /home/ubuntu/.cache/huggingface/datasets/csv/default-3aaa395a57a72ad0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-3705def8ad838715.arrow


Size of the train set: 18. Size of the validation set: 2


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.39s/it]
Using pad_token, but it is not set yet.


In [34]:
rlhf_step1_sft.train_and_save("../tests/step1_1708")

/opt/conda/envs/trl/lib/python3.10/site-packages/trl/trainer/utils.py:248: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/trl/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
